In [9]:
! pip install beautifulsoup4
! pip install lxml
! pip install geocoder

Step 1 - import necessary libraries

In [10]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [11]:
res =  requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]

Step 2 - Read excel datafile & display header

In [12]:
df = pd.read_html(str(table))[0]
df.head(10)
df.shape

(287, 3)

Step 3 - Ignore cells with a borough that is Not assigned

In [13]:
df = df[df['Borough']!='Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Step 4 - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [14]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
            row['Neighbourhood'] = row['Borough']

Step 5 - combine more than one neighborhood into one row with the neighborhoods separated with a comma

In [15]:
df = df.groupby(['Postcode'],as_index=False).agg({'Borough':'min','Neighbourhood':','.join})
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [16]:
df.shape

(103, 3)

Part 2: get the geographical coordinates (lat, long) for each postal code

In [17]:
#import coordinates into dataframe
df_coor = pd.read_csv('https://cocl.us/Geospatial_data')
df_coor.rename(columns={'Postal Code':'Postcode'}, inplace=True)
df_coor.head(10)

HTTPError: HTTP Error 503: Service Unavailable

In [ ]:
#Assign the coordinates to the postcodes
df_spatial = pd.merge(df,df_coor, left_index=False, right_index=False, sort=True)
df_spatial.head(10)